<a href="https://colab.research.google.com/github/Rewzaw/Torrent-to-GDrive-Downloader/blob/master/Torrent_to_GDrive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <h3>⬅️Run this Cell to  Mount Gdrive
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="110" alt="Gdrive logo"/></center>

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

In [ ]:
#@markdown <h3>⬅️ Run this Cell to Install Required Libraries 
#@markdown <br><center><img src='http://uupload.ir/files/iqwa_library.jpg' height="200" alt="Gdrive logo"/></center>

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
import libtorrent as lt
from IPython.display import HTML, clear_output

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

clear_output()

display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Successfully Configured!</h2><br></center>"))

# **For Personal Drive**

In [ ]:
#@markdown <h3>⬅️ Run this Cell to Upload Your .torrent File
#@markdown <h5>You can run this cell to add more files as many times as you want</h5>
#@markdown <h5>No parallel downloading using .torrent files, use magnet for that</h5><br>

from google.colab import files
from IPython.display import HTML, clear_output

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(source[fn])))
  
clear_output()

display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Successfully Uploaded!</h2><br></center>"))

In [ ]:
#@markdown <h3>⬅️ if You're Using Magnet Link, Run this Cell
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>

params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

In [ ]:
#@title
#@markdown <h3>⬅️ Run this Cell to Start Download</h3>
#@markdown <br><center><img src='http://uupload.ir/files/ikhz_download-synctoy-o.png' height="150" alt="Gdrive logo"/></center>

from IPython.display import HTML, clear_output
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)


print("""

                                                                  __________                               
                                                                  \______   \ ______  _  _______________   
                                                                  |       _// __ \ \/ \/ /\___   /\__  \  
                                                                  |    |   \  ___/\     /  /    /  / __ \_
                                                                  |____|___/\_____>\/\_/  /______\(______/        

        
""")

print("""
                                                                          Tel: t.me/PyC0der
""")

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Successfully Downloaded!</h2><br></center>"))

# **For Team Drive**

In [ ]:
#@title
#@markdown <h3>⬅️ Run this Cell to Upload Your .torrent File [for Team Drive] </h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5>
#@markdown <h5>Doesn't support parallel downloding, use magnet method for that</h5><br>

#@markdown <h3>Enter the path of TD :</h3>
path = "/content/drive/Shared drives/Google_Drive/Torrent" #@param {type:"string"}
check=[]


from google.colab import files
from IPython.display import HTML, clear_output

source = files.upload()
params = {
    "save_path": path,
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

clear_output()

display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Successfully Uploaded!</h2><br></center>"))

In [ ]:
#@markdown <h3>⬅️ if You're Using Magnet Link, Run this Cell [for Team Drive]
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>

#@markdown <h3>Enter the path of TD :</h3>
path = "/content/drive/Shared drives/Google_Drive/Torrent" #@param {type:"string"}
check=[]
params = {"save_path": path}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

clear_output()

print("Started Successfully")   

In [ ]:
#@title
#@markdown <h3>⬅️ Run this Cell to Start Download</h3>
#@markdown <br><center><img src='http://uupload.ir/files/ikhz_download-synctoy-o.png' height="150" alt="Gdrive logo"/></center>


from IPython.display import HTML, clear_output
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

print("""

                                                                  __________                               
                                                                  \______   \ ______  _  _______________   
                                                                  |       _// __ \ \/ \/ /\___   /\__  \  
                                                                  |    |   \  ___/\     /  /    /  / __ \_
                                                                  |____|___/\_____>\/\_/  /______\(______/        

        
""")

print("""
                                                                          Tel: t.me/PyC0der
""")

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Successfully Downloaded!</h2><br></center>"))